In this notebook, we will build a 3D map of a scene from a small set of images and then localize an image

In [10]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook, pycolmap
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval, localize_sfm, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d, io, parsers, database

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


use superpoint, superglue, exhaustive matching. 

In [ ]:
dir(pycolmap.Reconstruction())

In [4]:
images = Path('/home/keunmo/workspace/dataset/s20fe-robjet-uw-43/images/')
outputs = Path('/home/keunmo/workspace/dataset/s20fe-robjet-uw-43/hloc_output/')

sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_inloc']
matcher_conf = match_features.confs['superglue']

In [16]:
io.list_h5_names(features)[0]

'3.jpg'

In [18]:
len(io.get_keypoints(features, '3.jpg'))

2439

In [3]:
references = [p.relative_to(images).as_posix() for p in images.iterdir()]
print(len(references), "mapping images")
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

mapper_options = pycolmap.IncrementalMapperOptions()
mapper_options.ba_refine_focal_length = False
mapper_options.ba_refine_extra_params = False
mapper_options.ba_refine_principal_point = False
mapper_options = mapper_options.todict()

image_options = pycolmap.ImageReaderOptions()
image_options.camera_model = 'OPENCV' # fx, fy, cx, cy, k1, k2, p1, p2
image_options.camera_params = '1582.436430707376, 1579.117601109387, 1510.582207892258, 1988.252922889676, \
                                -0.0004238882159880583, 0.00007392309450114601, 0.01493723186454231, -0.01123193810676941'
# image_options.todict()

model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, camera_mode=pycolmap.CameraMode.SINGLE, image_list=references, image_options=image_options, mapper_options=mapper_options)

[2023/04/10 13:42:46 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}


54 mapping images
Loaded SuperPoint model


[2023/04/10 13:42:52 hloc INFO] Finished exporting features.
[2023/04/10 13:42:52 hloc INFO] Found 1431 pairs.
[2023/04/10 13:42:52 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


[2023/04/10 13:45:52 hloc INFO] Finished exporting matches.
[2023/04/10 13:45:52 hloc INFO] Creating an empty database...
[2023/04/10 13:45:52 hloc INFO] Importing images into the database...
[2023/04/10 13:45:57 hloc INFO] Importing features into the database...


  0%|          | 0/54 [00:00<?, ?it/s]

[2023/04/10 13:45:57 hloc INFO] Importing matches into the database...


  0%|          | 0/1431 [00:00<?, ?it/s]

[2023/04/10 13:45:57 hloc INFO] Performing geometric verification of the matches...
[2023/04/10 13:46:22 hloc INFO] Running 3D reconstruction...
[2023/04/10 13:47:12 hloc INFO] Reconstructed 1 model(s).
[2023/04/10 13:47:12 hloc INFO] Largest model is #0 with 54 images.
[2023/04/10 13:47:12 hloc INFO] Reconstruction statistics:
Reconstruction:
	num_reg_images = 54
	num_cameras = 1
	num_points3D = 14871
	num_observations = 40028
	mean_track_length = 2.69168
	mean_observations_per_image = 741.259
	mean_reprojection_error = 1.37832
	num_input_images = 54


use superpoint, superglue, netvlad retrieval matching. 

In [8]:
images = Path('/home/keunmo/workspace/dataset/s20fe-robjet-uw-43/images/')
outputs = Path('/home/keunmo/workspace/dataset/s20fe-robjet-uw-43/hloc_refine_output/')

sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_inloc']
matcher_conf = match_features.confs['superglue']

In [9]:
references = [p.relative_to(images).as_posix() for p in images.iterdir()]
print(len(references), "mapping images")
extract_features.main(feature_conf, images, image_list=references, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

# mapper_options = pycolmap.IncrementalMapperOptions()
# mapper_options.ba_refine_focal_length = False
# mapper_options.ba_refine_extra_params = False
# mapper_options.ba_refine_principal_point = False
# mapper_options = mapper_options.todict()

# image_options = pycolmap.ImageReaderOptions()
# image_options.camera_model = 'OPENCV' # fx, fy, cx, cy, k1, k2, p1, p2
# image_options.camera_params = '1578.536018839302, 1581.913683228172, 1988.052517109374, 1487.990041106547, \
#                                 -0.00007241601067241044, -0.0004441156330189372, 0.01447415647374481, -0.01074775857708951'
# image_options.todict()

model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, camera_mode=pycolmap.CameraMode.SINGLE, image_list=references)

[2023/04/07 17:13:44 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}


54 mapping images
Loaded SuperPoint model


[2023/04/07 17:13:47 hloc INFO] Finished exporting features.
[2023/04/07 17:13:47 hloc INFO] Found 1431 pairs.
[2023/04/07 17:13:47 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


[2023/04/07 17:16:51 hloc INFO] Finished exporting matches.
[2023/04/07 17:16:51 hloc INFO] Creating an empty database...
[2023/04/07 17:16:51 hloc INFO] Importing images into the database...
[2023/04/07 17:16:56 hloc INFO] Importing features into the database...


  0%|          | 0/54 [00:00<?, ?it/s]

[2023/04/07 17:16:56 hloc INFO] Importing matches into the database...


  0%|          | 0/1431 [00:00<?, ?it/s]

[2023/04/07 17:16:56 hloc INFO] Performing geometric verification of the matches...
[2023/04/07 17:17:25 hloc INFO] Running 3D reconstruction...
W0407 17:17:34.937197 145874 levenberg_marquardt_strategy.cc:116] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0407 17:17:34.939117 145874 levenberg_marquardt_strategy.cc:116] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0407 17:17:34.945113 145874 levenberg_marquardt_strategy.cc:116] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0407 17:17:34.946710 145874 levenberg_marquardt_strategy.cc:116] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W0407 17:17:34.954885 145874 levenberg_marquardt_strategy.cc:116] Linear solver failure. Failed to compute a step: Eigen failure. Unable to per

In [42]:
images = Path('datasets/box_recon/20230329_105301-wide-30fps')
outputs = Path('outputs/box_recon/20230329_105301-wide-30fps')

sfm_pairs = outputs / 'pairs-sfm.txt'
sfm_dir = outputs / 'sfm'
loc_pairs = outputs / 'pairs-loc.txt'

retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_inloc']
matcher_conf = match_features.confs['superglue']

In [43]:
references = [p.relative_to(images).as_posix() for p in (images / 'mapping/').iterdir()]
global_descriptors = extract_features.main(retrieval_conf, images, outputs, image_list=references)
feature_path = extract_features.main(feature_conf, images, outputs)
pairs_from_retrieval.main(global_descriptors, sfm_pairs, num_matched=15)
match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

mapper_options = pycolmap.IncrementalMapperOptions()
mapper_options.ba_refine_focal_length = False
mapper_options.ba_refine_extra_params = False
mapper_options.ba_refine_principal_point = False
mapper_options = mapper_options.todict()

image_options = pycolmap.ImageReaderOptions()
image_options.camera_model = 'OPENCV' # fx, fy, cx, cy, k1, k2, p1, p2
image_options.camera_params = '906.0804341080022, 911.9912443257355, 730.0029979627827, 700.6146374410489, 0.00003584909377177392, -0.00002014337127059466, -0.0099156764662087, 0.06144578903316186'
image_options.todict()

model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path, camera_mode=pycolmap.CameraMode.SINGLE, image_list=references, image_options=image_options, mapper_options=mapper_options)

[2023/03/31 10:37:52 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2023/03/31 10:37:58 hloc INFO] Finished exporting features.
[2023/03/31 10:37:58 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 4},
 'output': 'feats-superpoint-n4096-r1600',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}
[2023/03/31 10:37:58 hloc INFO] Found 51 images in root datasets/box_recon/20230329_105301-wide-30fps.


Loaded SuperPoint model


[2023/03/31 10:38:00 hloc INFO] Finished exporting features.
[2023/03/31 10:38:00 hloc INFO] Extracting image pairs from a retrieval database.
[2023/03/31 10:38:00 hloc INFO] Found 765 pairs.
[2023/03/31 10:38:00 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


[2023/03/31 10:38:26 hloc INFO] Finished exporting matches.
[2023/03/31 10:38:26 hloc INFO] Creating an empty database...
[2023/03/31 10:38:26 hloc INFO] Importing images into the database...
[2023/03/31 10:38:27 hloc INFO] Importing features into the database...


  0%|          | 0/51 [00:00<?, ?it/s]

[2023/03/31 10:38:27 hloc INFO] Importing matches into the database...


  0%|          | 0/765 [00:00<?, ?it/s]

[2023/03/31 10:38:27 hloc INFO] Performing geometric verification of the matches...
[2023/03/31 10:38:28 hloc INFO] Running 3D reconstruction...
[2023/03/31 10:39:02 hloc INFO] Reconstructed 1 model(s).
[2023/03/31 10:39:02 hloc INFO] Largest model is #0 with 51 images.
[2023/03/31 10:39:02 hloc INFO] Reconstruction statistics:
Reconstruction:
	num_reg_images = 51
	num_cameras = 1
	num_points3D = 10144
	num_observations = 38976
	mean_track_length = 3.84227
	mean_observations_per_image = 764.235
	mean_reprojection_error = 1.41348
	num_input_images = 51
